In [ ]:
import datetime
import ijson
import IPython
import hpq
import tabulate

conn = hpq.create_web_socket_client()
now = datetime.datetime.now(datetime.timezone.utc)
start = now - datetime.timedelta(minutes=20)
request = {
    "query": "ticks",
    "source": "cme",
    "product": "ESZ" + now.strftime("%Y")[3:],
    "start_time": start.strftime("%H:%M:%S"),
    "end_time": now.strftime("%H:%M:%S"),
    "time_zone": "UTC",
    "start_date": start.strftime("%Y-%m-%d"),
    "end_date": now.strftime("%Y-%m-%d"),
    "messages": ["TRD", "ADD", "MOD", "LVL", "STA"],
}
ticks = []
no_follow = None
count = 20
actual = 0
for e in ijson.items(conn.stream(request), "item"):
    if e["type"] == "no_follow":
        no_follow = e
        break
    if e["type"] == "trade" and not e["printable"]:
        continue
    actual += 1
    if len(ticks) > count:
        ticks.pop(0)
    ticks.append(e)
start_formatted = start.strftime("%Y-%m-%dT%H:%M:%SZ")
now_formatted = now.strftime("%Y-%m-%dT%H:%M:%SZ")
IPython.display.display(
    IPython.display.HTML(f"<h1>{start_formatted} to {now_formatted}</h1>")
)
IPython.display.display(
    IPython.display.HTML(
        f"<p>Most recent {count} results shown (actual result set is {actual} results)</p>"
    )
)
if no_follow is not None:
    up_to_formatted = hpq.format_timestamp(no_follow["most_recent_known_timestamp"])
    IPython.display.display(
        IPython.display.HTML(
            f"<p>Due to near time nature of HPQ only data up to {up_to_formatted} (inclusive) is available, query again later for more recent ticks</p>"
        )
    )
ticks = map(hpq.format, ticks)
table = tabulate.tabulate(ticks, tablefmt="html", headers="keys")
IPython.display.display(IPython.display.HTML(table))